
# ***User_features***






In [1]:
import pandas as pd
# Display all columns
pd.set_option('display.max_columns', None)

# Display all rows
pd.set_option('display.max_rows', None)

In [2]:
import urllib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import PIL
import os
from pathlib import Path




In [3]:
#from google.colab import auth
#auth.authenticate_user()
#from google.cloud import bigquery
#from pandas_gbq import read_gbq


#project_id = "data-analytics-bootcamp-363212"

#query = '''
#SELECT *
#FROM `data-analytics-bootcamp-363212.neo_bank.transactions`
#'''

#df_transactions = read_gbq(query, project_id=project_id)

In [4]:
#os.makedirs("/content/drive/MyDrive/Documents", exist_ok=True)

#df_transactions.to_csv(
 #   "/content/drive/MyDrive/Documents/eval_transactions.csv",
  #  index=False)

#print("Saved to Google Drive Documents folder")

In [5]:
#df_transactions.to_csv("eval_transactions.csv", index=False)

#from google.colab import files
#files.download("eval_transactions.csv")

In [6]:
#from google.colab import drive
#drive.mount('/content/drive')

In [7]:
df_transactions = pd.read_csv(
    "eval_transactions.csv"
)

df_transactions.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,user_id,created_date
0,transaction_505,TOPUP,AED,298.62,COMPLETED,NaN,NaN,NaN,NaN,INBOUND,user_4773,2018-08-02 09:41:07.426363+00:00
1,transaction_1755379,EXCHANGE,AED,3.51,COMPLETED,NaN,NaN,NaN,NaN,OUTBOUND,user_10435,2018-09-06 10:23:15.807148+00:00
2,transaction_1667,EXCHANGE,AED,0.05,COMPLETED,NaN,NaN,NaN,NaN,OUTBOUND,user_397,2018-05-16 06:09:28.063037+00:00
3,transaction_1429,EXCHANGE,AED,24.14,COMPLETED,NaN,NaN,NaN,NaN,OUTBOUND,user_4399,2018-12-19 20:22:03.550130+00:00
4,transaction_345,EXCHANGE,AED,0.21,COMPLETED,NaN,NaN,NaN,NaN,OUTBOUND,user_1731,2018-10-27 13:29:56.524826+00:00


In [8]:
df_transactions.columns = df_transactions.columns.str.lower().str.replace(" ", "_")
text_cols = df_transactions.select_dtypes(include='object').columns

for col in text_cols:
    df_transactions[col] = (df_transactions[col].str.lower() .str.strip().str.replace(r'\s+', '_', regex=True) .str.replace(r'[^\w]', '', regex=True))

In [9]:
df_transactions["ea_merchant_city"] = (
    df_transactions["ea_merchant_city"]
    # 1) fake NaNs
    .replace(
        ["nan", "Nan", "NAN", "none", "None", "null", "NULL", "", " "],
        pd.NA
    )
    .fillna("Not Applicable")
    # 2) cleaning
    .astype(str)
    .str.lower()
    .str.replace(r'\b\d{2}-\d{3}\b', '', regex=True)
    .str.replace(r'[_\d]+', ' ', regex=True)
    .str.replace(r'^\s*-+', '', regex=True)
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
    .str.title()
    # 3) numeric-only OR empty after cleaning → Not Applicable
    .replace(r'^\d+$', "Not Applicable", regex=True)
    .replace(r'^$', "Not Applicable", regex=True)
)


In [10]:
!pip install pycountry

In [11]:
import pycountry

def alpha3_to_alpha2(code):
    try:
        return pycountry.countries.get(alpha_3=code).alpha_2
    except:
        return None

df_transactions["merchant_country"] = (
    df_transactions["ea_merchant_country"]
    .apply(alpha3_to_alpha2)
)

In [12]:
df_transactions['ea_merchant_country']=df_transactions["ea_merchant_country"].str.upper()

In [13]:
df_transactions["ea_merchant_city"] = df_transactions["ea_merchant_city"].fillna("Not Applicable")
df_transactions["merchant_country"] = df_transactions["merchant_country"].fillna("Not Applicable")
df_transactions["ea_merchant_country"] = df_transactions["ea_merchant_country"].fillna("Not Applicable")
df_transactions["ea_cardholderpresence"] = df_transactions["ea_cardholderpresence"].fillna("Not Applicable")

In [14]:
df_transactions["ea_merchant_mcc"] = df_transactions["ea_merchant_mcc"].fillna(-1)

In [15]:
df_transactions[df_transactions.duplicated(keep=False)]

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,user_id,created_date,merchant_country


In [16]:
#df_transactions["amount_usd"] = df_transactions["amount_usd"].clip(upper=100)

In [17]:
#df_transactions = df_transactions[df_transactions["amount_usd"] < 70]

In [18]:
#CLEANING THE OUTLIERS

#Q1 = df_transactions["amount_usd"].quantile(0.25)
#Q3 = df_transactions["amount_usd"].quantile(0.75)
#IQR = Q3 - Q1

#lower_bound = Q1 - 1.5 * IQR
#upper_bound = Q3 + 1.5 * IQR

#lower_bound, upper_bound

#df_transactions_clean = df_transactions[
 #   (df_transactions["amount_usd"] >= lower_bound) &
 #   (df_transactions["amount_usd"] <= upper_bound)]

 #sns.boxplot(x=df_transactions_clean["amount_usd"])
#plt.title("Transaction Amount")
#plt.show()




In [19]:

# =========================
# 1. IQR calculation
# =========================
Q1 = df_transactions["amount_usd"].quantile(0.25)
Q3 = df_transactions["amount_usd"].quantile(0.75)
IQR = Q3 - Q1

lower_1 = Q1 - 1.5 * IQR
upper_1 = Q3 + 1.5 * IQR

lower_2 = Q1 - 3 * IQR
upper_2 = Q3 + 3 * IQR

# =========================
# 2. Create NEW dataframe
# =========================

df_transactions["outlier_level"] = "normal"

df_transactions.loc[
    (df_transactions["amount_usd"] < lower_1) | (df_transactions["amount_usd"] > upper_1),
    "outlier_level"
] = "mild"

df_transactions.loc[
    (df_transactions["amount_usd"] < lower_2) | (df_transactions["amount_usd"] > upper_2),
    "outlier_level"
] = "extreme"


In [20]:
df_transactions['created_date'] = (
    df_transactions['created_date']
    .astype(str)
    .str.extract(r'(\d{8}_\d{6})')[0]
)

df_transactions['created_date'] = pd.to_datetime(
    df_transactions['created_date'],
    format='%Y%m%d_%H%M%S',
    errors='coerce'
)

In [21]:
df_completed = df_transactions[
    df_transactions['transactions_state'] == 'completed'
]
user_features = df_completed.groupby('user_id').agg(
    total_transactions=('transaction_id','count'),
    avg_transaction_amount=('amount_usd','mean'),
    last_transaction_date=('created_date','max')
).reset_index()

In [22]:
reference_date = df_completed['created_date'].max()

In [23]:
user_features['days_since_last_transaction'] = (
    reference_date - user_features['last_transaction_date']
).dt.days

In [24]:
user_activity_span = df_completed.groupby('user_id')['created_date'].agg(['min','max']).reset_index()

user_activity_span['activity_days'] = (
    user_activity_span['max'] - user_activity_span['min']
).dt.days + 1

In [25]:
user_features = user_features.merge(
    user_activity_span[['user_id','activity_days']],
    on='user_id',
    how='left'
)

In [26]:
user_features['transaction_frequency'] = (
    user_features['total_transactions'] /
    user_features['activity_days']
)

In [27]:
user_features['churn_flag']=(user_features['days_since_last_transaction']>30).astype(int)

In [28]:
user_features.head()

,user_id,total_transactions,avg_transaction_amount,last_transaction_date,days_since_last_transaction,activity_days,transaction_frequency,churn_flag
0,user_0,350,36.138857,2019-05-02 03:38:44,13,471,0.743100,0
1,user_1,114,50.046930,2019-02-24 21:25:15,79,390,0.292308,1
2,user_10,846,28.969716,2019-05-11 11:01:00,4,493,1.716024,0
3,user_100,110,146.419818,2019-05-07 13:00:24,8,477,0.230608,0
4,user_1000,35,575.094571,2018-06-26 05:00:42,323,171,0.204678,1


In [29]:
user_features.to_csv("user_features.csv",index=False,encoding="utf-8")

from google.colab import files
files.download("user_features.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>